# Walkthrough

This notebook provides a short walkthrough of some of the features of the `sharrow` library.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from io import StringIO

import sharrow as sh
sh.__version__

In [ ]:
# check versions
import packaging
assert packaging.version.parse(sh.__version__) >= packaging.version.parse("2022.0")
assert packaging.version.parse(xr.__version__) >= packaging.version.parse("0.20.2")

## Example Data

We'll begin by importing some example data to work with.  We'll be using 
some test data taken from the MTC example in the ActivitySim project, including 
tables of data for households and persons, as well as a set of 
skims containing transportation level of service information for travel around
a tiny slice of San Francisco.

The households and persons are typical tabular data, and 
each can be read in and stored as a `pandas.DataFrame`.

In [ ]:
households = sh.example_data.get_households()
households.head()

In [ ]:
# test households content
assert len(households) == 5000
assert "income" in households 
assert households.index.name == "HHID"

In [ ]:
persons = sh.example_data.get_persons()
persons.head()

In [ ]:
assert len(persons) == 8212
assert "household_id" in persons
assert persons.index.name == 'PERID'

The skims, on the other hand, are not just simple tabular data, but rather a 
multi-dimensional representation of the transportation system, indexed by origin.
destination, and time of day. Rather than using a single DataFrame for this data,
we store it as a multi-dimensional `xarray.Dataset` — or, more exactly, a 
`sharrow.Dataset`, which is a subclass from the xarray version that adds some 
useful features we'll see later.

In [ ]:
skims = sh.example_data.get_skims()
skims

Suppose we're wanting to simulate a tour mode choice.  Normally we'd probably have
run through a bunch of different models to generate these tours and their destinations
first, but let's just skip that for now and make up some random data to work with.  We'll 
just randomly choose (with replacement) 100,000 people, and send them to 100,000 zones, with
random outbound and inbound time periods.

In [ ]:
def random_tours(n_tours=100_000, seed=42):
    rng = np.random.default_rng(seed)
    n_zones = skims.dims['dtaz']
    return pd.DataFrame({
        'PERID': rng.choice(persons.index, size=n_tours),
        'dest_taz_idx': rng.choice(n_zones, size=n_tours),
        'out_time_period': rng.choice(skims.time_period, size=n_tours),
        'in_time_period': rng.choice(skims.time_period, size=n_tours),
    }).rename_axis("TOURIDX")
tours = random_tours()
tours.head()

In [ ]:
assert tours.index.name == "TOURIDX"
assert 0 in tours.head().dest_taz_idx

Of note in this table, we include include destination TAZ's by index (position) not 
label, so we can observe a TAZ index of `0` even though the first TAZ ID is 1.

## Spec Files

Now that we've got our tours to work with, we'll also need 
an expression "spec" file that defines the utility function
terms and coefficients.  Following the ActivitySim format, we
can write a mini-spec file as appears below.  Each line of this
CSV file has an expression that can be evaluated in the context
of the various tables and datasets shown above, plus a set of 
coefficients that apply for that expression across various modal 
alternatives (drive, walk, and transit in this example).

In [ ]:
mini_spec = """
Label,Expression,DRIVE,WALK,TRANSIT
Drive Time,odt_skims['SOV_TIME'] + dot_skims['SOV_TIME'],-0.0134,,
Transit IVT,(odt_skims['WLK_LOC_WLK_TOTIVT']/100 + dot_skims['WLK_LOC_WLK_TOTIVT']/100),,,-0.0134
Transit Wait Time,short_i_wait_mult * ((odt_skims['WLK_LOC_WLK_IWAIT']/100).clip(upper=shortwait) + (dot_skims['WLK_LOC_WLK_IWAIT']/100).clip(upper=shortwait)),,,-0.0134
Income,hh.income > 60000,,-0.2,
Constant,1,,-0.4,-0.55
"""

We'll use pandas to load these values into a DataFrame.

In [ ]:
spec = pd.read_csv(StringIO(mini_spec), index_col='Label')
spec

In [ ]:
assert spec.index.name == "Label"
assert all(spec.columns == ['Expression', 'DRIVE', 'WALK', 'TRANSIT'])

## Data Trees and Flows

Then, it's time to prepare our data.  We'll create a `DataTree`
that defines the relationships among all the datasets we're working
with.  This is a tree in the mathematical sense, with nodes referencing
the datasets and edges representing the relationships.

In [ ]:
tree_mc = sh.DataTree(
    tour=tours,
    person=persons,
    hh=households,
    odt_skims=skims,
    dot_skims=skims,
    relationships=(
        "tour.PERID @ person.PERID",
        "person.household_id @ hh.HHID",
        "hh.TAZ @ odt_skims.otaz",
        "tour.dest_taz_idx -> odt_skims.dtaz",
        "tour.out_time_period @ odt_skims.time_period",
        "tour.dest_taz_idx -> dot_skims.otaz",
        "hh.TAZ @ dot_skims.dtaz",
        "tour.in_time_period @ dot_skims.time_period",
    ),
    extra_vars={
        'short_i_wait_mult': 0.75,
        'shortwait': 3.0,
    },
)

The first named dataset we include, `tour`, is by default the root node of this data tree.
We then can define an arbitrary number of other named data nodes.  Here, we add `person`, `hh`,
`odt_skims` and `odt_skims`.  Note that these last two are actually two different names for the
same underlying dataset, and for each name we will next define a unique set of relationships.

All data nodes in this tree are stored as `Dataset` objects. We can give a pandas DataFrame
in this contructor instead, but it will be automatically converted into a one-dimension `Dataset`.
The conversion is no-copy if possible (and it is usually possible) so no additional memory is
consumed in the conversion.

The `relationships` defines links of the data tree. Each relationship maps a particular variable
in a named upstream dataset to a particular dimension of a named downstream dataset.  For example,
`"person.household_id @ hh.HHID"` tells the tree that the `household_id` variable in the `person` 
dataset contains labels (`@`) that map to the `HHID` dimension of the `hh` dataset.

In addition to mapping by label, we can also map by position, by using the `->` operator in the
relationship string instead of `@`.  In the example above, we map the tour destination TAZ's in
this manner, as the `dest_taz_idx` variable in the `tours` dataset contains positional references
instead of labels.

Lastly, out tree definition includes a few named constants, that are just fixed values defined
in a seperate dictionary.

Once we have defined our data tree, we can use it along with the `spec`, to compute the utility
for various alternatives in the choice model.  Sharrow allows us to compile this utility function
into a `Flow`, which can be reused for massive speed gains on later utility evaluations.

In [ ]:
flow_mc = tree_mc.setup_flow(spec.Expression)

To use a `Flow` for preparing the array of data that backs the utility
function, we can call the `load()` method. The first time we call `load()`,
it takes a (relatively) long time to evaluate, as the expressions are compiled
and that compiled code is cached to disk.

In [ ]:
%time flow_mc.load()

In [ ]:
# test utility data
actual = flow_mc.load()
expected = np.array([[  9.4     ,  16.9572  ,   4.5     ,   0.      ,   1.      ],
       [  9.32    ,  14.3628  ,   4.5     ,   1.      ,   1.      ],
       [  7.62    ,  11.0129  ,   4.5     ,   1.      ,   1.      ],
       [  4.25    ,   7.6692  ,   2.50065 ,   0.      ,   1.      ],
       [  6.16    ,   8.2186  ,   3.387825,   0.      ,   1.      ],
       [  4.86    ,   4.9288  ,   4.5     ,   0.      ,   1.      ],
       [  1.07    ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  8.52    ,  11.615499,   3.260325,   0.      ,   1.      ],
       [ 11.74    ,  16.2798  ,   3.440325,   0.      ,   1.      ],
       [ 10.48    ,  13.3974  ,   3.942825,   0.      ,   1.      ]], dtype=np.float32)

np.testing.assert_array_almost_equal(actual[:5], expected[:5])
np.testing.assert_array_almost_equal(actual[-5:], expected[-5:])
assert actual.shape == (len(tours), len(spec))

Subsequent calls to `load()` are much faster.

In [ ]:
%time flow_mc.load()

It's not faster because it's cached the data, but because it's cached the compiled code.
We can swap out the `tour` node in the tree for a different set of (similarly formatted)
tours, and re-evaluate at that fast speed.

In [ ]:
tours_2 = random_tours(seed=43)
tours_2.head()

Note that the flow requires not just a base dataset but a whole DataTree to operate,
so to re-evaluate with a new `tours` we need to make a DataTree with `replace_datasets`.
Fortuntately, this operation is no-copy so it doesn't consume much memory.  If all the 
datasets in a tree are linked by position (instead of by label) this would be almost 
instantaneous, but since our example tree here has tours linked by label it takes just a
moment to rebuild the linkages.

In [ ]:
tree_2 = tree_mc.replace_datasets(tour=tours_2)

In [ ]:
%time flow_mc.load(tree_2)

The load function also has some other features, like nicely formatting the output
into a DataFrame.

In [ ]:
df = flow_mc.load_dataframe()
df

In [ ]:
# test df
assert len(df) == len(tours)
pd.testing.assert_index_equal(
    df.columns, 
    pd.Index(['Drive Time', 'Transit IVT', 'Transit Wait Time', 'Income', 'Constant']),
)
expected_df_head = pd.read_csv(StringIO(''',Drive Time,Transit IVT,Transit Wait Time,Income,Constant
0,9.4,16.9572,4.5,0.0,1.0
1,9.32,14.3628,4.5,1.0,1.0
2,7.62,11.0129,4.5,1.0,1.0
3,4.25,7.6692,2.50065,0.0,1.0
4,6.16,8.2186,3.387825,0.0,1.0'''), index_col=0).astype(np.float32)
pd.testing.assert_frame_equal(df.head(), expected_df_head)

## Linear-in-Parameters Functions

When the `spec` represents a linear-in-parameters utility function, the data 
we get out of the `load()` function represents one matrix in a dot-product, and
the coefficients in the `spec` provide the other matrix.  We might look to 
use the efficient linear algebra algorithms embedded in `np.dot` to compute the
utility, like this:

In [ ]:
x = flow_mc.load()
b = spec.iloc[:,1:].fillna(0).astype(np.float32).values
np.dot(x, b)

But `sharrow` provides a substantially faster option, by embedding
the dot product directly into the compiled code and never instantiating the
full `x` array in memory at all.

In [ ]:
%time u = flow_mc.dot(b)
u

In [ ]:
# test utility
np.testing.assert_array_almost_equal(u, np.dot(x, b))

As before, the compiler runs only the first time we apply the this 
function with this structure, and subsequent runs are faster, even with
different source data.

In [ ]:
%time flow_mc.dot(b, source=tree_2)

As for the plain `load` method, the `dot` method also has some formatted output versions.
For example, the `dot_dataarray` returns a `DataArray`.

In [ ]:
flow_mc.dot_dataarray(b, source=tree_2)

It works better if the coefficients are given as a DataArray too, so it 
can harvest dimension names and coordinates as appropriate.

In [ ]:
B = xr.DataArray(
    spec.iloc[:,1:].fillna(0).astype(np.float32), 
    dims=('expressions','modes')
)
flow_mc.dot_dataarray(B, source=tree_2)

## Multinomial Logit Simulation

The next level of flow evaluation is made by treating the dot-product as a
linear-in-parameters multinomial logit (MNL) utility function, and making simulated
choices based on that model.  To do this, we'll need to provide the random
draws as a function input (which also lets us attach any randomization engine
we prefer, e.g. a reproducible random generator).  For this example, we'll 
create one random (uniform) draw for each tour.

In [ ]:
draws_mc = np.random.default_rng(321).random(size=tree_mc.shape[0])

Given those draws, we use the `mnl_draws` method to build and apply a 
MNL simulator, which returns to us both the choices and the probability that
was computed for each chosen alternative.

In [ ]:
choices_mc, choice_probs_mc = flow_mc.mnl_draws(b, draws_mc)
choices_mc, choice_probs_mc

In [ ]:
# test mnl choices
uz = np.exp(flow_mc.dot(b))
uz = uz / uz.sum(1)[:,None]
np.testing.assert_array_almost_equal(
    uz[range(uz.shape[0]),choices_mc.ravel()],
    choice_probs_mc.ravel(),
)

## Multi-Dimensional Analysis

Let take a look at preparing some data for a workplace location choice simulation model.
This is a different kind of model, and it will use differently shaped data. The decision 
makers (or "choosers") in this model will be the workers. The alternatives 
will be the various zones included in the land use table.

We didn't use the land use data previosly, so let's load it now.

In [ ]:
landuse = sh.example_data.get_land_use()
landuse.head()

As noted above, the decision makers (or "choosers") in this model will be workers, 
who are only a subset of the persons data we looked at before.  We can identify workers from
values 1 and 2 (full-time employed and part-time employed) in the `'pemploy'` attribute 
of the `persons` table. 

As we filter the persons table to just the workers, we will also rename the index from
"PERSONID" to "WORKERID".  This renaming is important for sharrow, as it expects dimensions
that have the same name to match, but the workers don't align directly with the persons 
anymore.

In [ ]:
workers = persons.query("pemploy in [1,2]").rename_axis(index='WORKERID')
workers

For our workplace location choice model, we will want to link in data from our skims,
which can tell us about travel times and costs.  Since we have not yet determined a 
time of day for each worker's work tours, we'll just use the `'AM'` skims for the outbound
leg of a hypothetical work tour, and the `'PM'` skims for the return leg.

In [ ]:
skims_am = skims.sel(time_period='AM')
skims_pm = skims.sel(time_period='PM')

## Creating a DataTree Iteratively

The last step in getting ready for this model is building out the relationships between all
this data we've prepared. We'll again use the `DataTree` class to do that, but this time 
we'll demostrate building the tree in stages.  First, we'll assign a
base Dataset to be the root data for the tree. We can start by creating an otherwise empty `Dataset` indexed on the two dimensions we want to end up with for analysis, workers and zones.

In [ ]:
base = sh.Dataset.from_named_objects(
    workers.index, 
    landuse.index,
)

Since our base dataset has
two dimensions, we can specify a dimension order when writing into
a DataTree (the default is alphabetical order).
This ordering will be applied to outputs from the flows later.

In [ ]:
tree_dest = sh.DataTree(base=base, dim_order=('WORKERID', 'TAZ'))

In [ ]:
# test tree_dest attributes
assert tree_dest.dim_order == ('WORKERID', 'TAZ')
assert tree_dest.shape == (4361, 25)

Then, we can progressively build our `DataTree` by adding additional data. 
Each new branch of the tree we want to add using the `add_dataset` command should have a 
unique name, a dataset being attached, and one or more relationship declarations
that describe how the new data attaches.  For example, we can attach the `persons`
data like this:

In [ ]:
tree_dest.add_dataset('person', persons, "base.WORKERID @ person.PERID")

The relationship definition here starts with a dotted name of some data 
dimension already in the tree, an `@` operator to indicating matching by
label in that dimension.

In [ ]:
tree_dest.add_dataset('landuse', landuse, "base.TAZ @ landuse.TAZ")
tree_dest.add_dataset('hh', households, "person.household_id @ hh.HHID")

Unlike in the mode choice example above, we've already filtered the 
time period dimensions of the skims to be morning and afternoon peak,
so we simply attach the two different parts, linking relationships only
for the remaining dimensions.

In [ ]:
tree_dest.add_dataset(
    'odskims', 
    skims_am, 
    relationships=(
        "hh.TAZ @ odskims.otaz", 
        "base.TAZ @ odskims.dtaz",
    ),
)

tree_dest.add_dataset(
    'doskims', 
    skims_pm, 
    relationships=(
        "base.TAZ @ doskims.otaz",
        "hh.TAZ @ doskims.dtaz",
    ),
)

## Dynamically Defined Flows 

Although it is convenient, especially when working with 
ActivitySim, it's not strictly necessary to employ a 'spec' file in csv format; 
a simple Python dictionary can also be used to setup a flow.

In [ ]:
flow_dc = tree_dest.setup_flow({
    'round_trip_dist': 'odskims.DIST + doskims.DIST',
    'round_trip_dist_first_mile': 'clip(odskims.DIST, 0, 1) + clip(doskims.DIST, 0, 1)',
    'round_trip_dist_addl_miles': 'clip(odskims.DIST-1, 0, None) + clip(doskims.DIST-1, 0, None)',
})

Loading from this flow is done the same as before.

In [ ]:
arr = flow_dc.load()
arr

In [ ]:
assert arr.shape == (4361, 25, 3)
expected = np.array([[[ 0.61,  0.61,  0.  ],
        [ 0.28,  0.28,  0.  ],
        [ 0.56,  0.56,  0.  ],
        [ 0.53,  0.53,  0.  ],
        [ 1.23,  1.23,  0.  ]],

       [[ 1.19,  1.19,  0.  ],
        [ 1.49,  1.49,  0.  ],
        [ 1.88,  1.85,  0.03],
        [ 1.36,  1.36,  0.  ],
        [ 1.93,  1.93,  0.  ]],

       [[ 1.19,  1.19,  0.  ],
        [ 1.49,  1.49,  0.  ],
        [ 1.88,  1.85,  0.03],
        [ 1.36,  1.36,  0.  ],
        [ 1.93,  1.93,  0.  ]],

       [[ 0.24,  0.24,  0.  ],
        [ 0.61,  0.61,  0.  ],
        [ 1.01,  1.01,  0.  ],
        [ 0.75,  0.75,  0.  ],
        [ 1.38,  1.38,  0.  ]],

       [[ 0.61,  0.61,  0.  ],
        [ 0.28,  0.28,  0.  ],
        [ 0.56,  0.56,  0.  ],
        [ 0.53,  0.53,  0.  ],
        [ 1.23,  1.23,  0.  ]]], dtype=np.float32)

np.testing.assert_array_almost_equal(arr[:5, :5, :], expected)

For the tour mode example above, the tours dataset had only one dimension (TOURIDX),
and so the output of the load function had two dimensions (TOURIDX and expressions).
In this example, the base dataset in the tree has two dimensions (workers and zones)
and so the result from the basic `load` function has *three* dimensions (workers, zones, and expressions).

In [ ]:
arr.shape

Just as we could neatly format the two-dimensional output above as a `pandas.DataFrame`,
so too can we neatly format this three-dimensional output, as a `xarray.DataArray`.

In [ ]:
arr_pretty = flow_dc.load_dataarray()
arr_pretty

In [ ]:
assert isinstance(arr_pretty, xr.DataArray)
assert arr_pretty.dims == ('WORKERID', 'TAZ', 'expressions')
assert arr_pretty.shape == (4361, 25, 3)
assert all(arr_pretty.expressions == np.array(['round_trip_dist', 'round_trip_dist_first_mile',
       'round_trip_dist_addl_miles'], dtype='<U26'))

## Dot-products with More Dimensions

We can also use the `dot` method here with the two dimensional base.
We'll apply a one-dimensional coefficients array, with length three to 
match the three terms in the spec.

In [ ]:
coefs = np.asarray([1.0, 0.1, 0.01])
flow_dc.dot(coefs)

The `dot_dataarray` method does the same underlying computational work, but 
yields a well-formatted DataArray intead of just a plain numpy array.

In [ ]:
flow_dc.dot_dataarray(coefs)

## More Logit Simulation

And, just as above, we can build and simulate an MNL model directly.

In [ ]:
draws_dc = np.random.default_rng(123).random(size=[4361, 4])

In [ ]:
dc_choices, dc_choice_probs = flow_dc.mnl_draws(
    coefficients=coefs,
    draws=draws_dc,
)

In [ ]:
dc_choices

In [ ]:
dc_choice_probs

In [ ]:
expected = np.array(
    [  [ 5,  8,  9, 18],
       [ 6,  8, 19, 23],
       [ 7,  9, 20, 22],
       [ 8, 18, 18, 19],
       [ 9, 17, 19, 20],
       # ...,
       [ 1, 12, 21, 22],
       [ 0,  4, 21, 23],
       [ 0,  3, 21, 21],
       [ 3,  6, 22, 23],
       [14, 14, 22, 23]], dtype=np.int32)
np.testing.assert_array_equal(dc_choices[:5], expected[:5])
np.testing.assert_array_equal(dc_choices[-5:], expected[-5:])

expected = np.array([[ 0.021751,  0.081996,  0.09071 ,  0.239192],
       [ 0.052467,  0.139753,  0.064863,  0.03138 ],
       [ 0.063566,  0.103221,  0.021153,  0.05463 ],
       [ 0.084491,  0.246472,  0.246472,  0.134457],
       [ 0.09071 ,  0.082828,  0.130486,  0.038443],
       ##...,
       [ 0.039684,  0.020115,  0.087248,  0.207961],
       [ 0.046644,  0.018624,  0.087248,  0.069162],
       [ 0.046644,  0.025064,  0.087248,  0.087248],
       [ 0.025064,  0.027071,  0.207961,  0.069162],
       [ 0.048079,  0.048079,  0.207961,  0.069162]])
np.testing.assert_array_almost_equal(dc_choice_probs[:5], expected[:5])
np.testing.assert_array_almost_equal(dc_choice_probs[-5:], expected[-5:])


It's more common to make many repeated choices for destination choice type models
(e.g. to sample destinations), so there's also a "pick count" feature, that
can summarize the simulation results efficiently.

In [ ]:
dc_choices_, dc_choice_probs_, dc_pick_count = flow_dc.mnl_draws(
    coefficients=coefs,
    draws=draws_dc,
    pick_counted=True,
)

If you compare against the non-pick-counted results above, you'll see 
that we get exactly the same choices out, but when choices are repeated
they are aggregated in the resulting arrays.

In [ ]:
np.concatenate([dc_choices_, dc_pick_count], axis=1)

In [ ]:
expected = np.array([[ 5,  8,  9, 18,  1,  1,  1,  1],
       [ 6,  8, 19, 23,  1,  1,  1,  1],
       [ 7,  9, 20, 22,  1,  1,  1,  1],
       [ 8, 18, 19, -1,  1,  2,  1,  0],
       [ 9, 17, 19, 20,  1,  1,  1,  1],
       #...,
       [ 1, 12, 21, 22,  1,  1,  1,  1],
       [ 0,  4, 21, 23,  1,  1,  1,  1],
       [ 0,  3, 21, -1,  1,  1,  2,  0],
       [ 3,  6, 22, 23,  1,  1,  1,  1],
       [14, 22, 23, -1,  2,  1,  1,  0]], dtype=np.int32)
y = np.concatenate([dc_choices_, dc_pick_count], axis=1)
np.testing.assert_array_equal(y[:5], expected[:5])
np.testing.assert_array_equal(y[-5:], expected[-5:])
